In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import f1_score

In [26]:
#Construccion de una funcion que realice el particionado completo
def train_val_test_split(df, rstate=42, shuffle=True, stratify=None):
    strat = df[stratify] if stratify else None
    train_set, test_set = train_test_split(
        df, test_size=0.4, random_state=rstate, shuffle=shuffle, stratify=strat
    )
    strat = test_set[stratify] if stratify else None
    val_set, test_set = train_test_split(
        test_set, test_size=0.5, random_state=rstate, shuffle=shuffle, stratify=strat
    )
    return (train_set, val_set, test_set)

In [27]:
def remove_labels(df, label_name):
    x = df.drop(label_name, axis=1)
    y = df[label_name].copy()
    return (x, y)

Lectura del conjunto de datos

In [28]:
df = pd.read_csv('urls.csv')

Visualizacion del conjunto de datos

In [29]:
df.head(10)

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
0,0,2,12,5.5,8,4.083334,2,15,7,0,...,-1,-1,-1,0.676804,0.860529,-1.000000,-1.000000,-1.00000,-1.000000,benign
1,0,3,12,5.0,10,3.583333,3,12,8,2,...,1,0,-1,0.715629,0.776796,0.693127,0.738315,1.00000,-1.000000,benign
2,2,2,11,4.0,5,4.750000,2,16,11,0,...,2,0,1,0.677701,1.000000,0.677704,0.916667,0.00000,0.898227,benign
3,0,2,7,4.5,7,5.714286,2,15,10,0,...,0,0,-1,0.696067,0.879588,0.818007,0.753585,0.00000,-1.000000,benign
4,19,2,10,6.0,9,2.250000,2,9,5,0,...,5,4,3,0.747202,0.833700,0.655459,0.829535,0.83615,0.823008,benign
5,0,2,10,5.5,9,4.100000,2,15,11,0,...,-1,-1,-1,0.732981,0.860529,-1.000000,-1.000000,-1.00000,-1.000000,benign
6,0,2,12,4.5,6,5.333334,2,24,9,0,...,0,0,-1,0.692383,0.939794,0.910795,0.673973,0.00000,-1.000000,benign
7,0,2,11,3.5,4,3.909091,2,15,6,0,...,0,0,-1,0.707365,0.916667,0.916667,0.690332,0.00000,-1.000000,benign
8,0,2,9,2.5,3,4.555555,2,6,3,0,...,1,0,-1,0.742606,1.000000,0.785719,0.808833,1.00000,-1.000000,benign
9,0,2,13,4.5,6,5.307692,2,16,9,1,...,-1,-1,-1,0.734633,0.939794,-1.000000,-1.000000,-1.00000,-1.000000,benign


In [30]:
df.describe()

C:\Users\arsms\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_Directoryname,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath
count,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15096.000000,15367.000000,15367.000000,15367.000000,15367.000000,...,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,15367.000000,13541.000000,15177.000000,15364.000000,15364.000000
mean,3.446021,2.543698,8.477061,5.851956,10.027461,5.289936,2.543698,12.659986,8.398516,1.910913,...,2.120843,1.124618,0.500813,-0.158782,0.721684,0.854232,0.634859,0.682896,0.313617,-0.723793
std,14.151453,0.944938,4.660250,2.064581,5.281090,3.535097,0.944938,8.562206,6.329007,4.657731,...,2.777307,2.570246,2.261013,2.535939,0.049246,0.072641,0.510992,0.502288,0.576910,0.649785
min,0.000000,2.000000,0.000000,1.500000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,0.419560,0.561913,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.000000,2.000000,5.000000,4.500000,7.000000,3.800000,2.000000,6.000000,4.000000,0.000000,...,1.000000,0.000000,0.000000,-1.000000,0.687215,0.798231,0.709532,0.707165,0.000000,-1.000000
50%,0.000000,2.000000,8.000000,5.500000,9.000000,4.500000,2.000000,11.000000,7.000000,0.000000,...,2.000000,0.000000,0.000000,-1.000000,0.723217,0.859793,0.785949,0.814038,0.000000,-1.000000
75%,0.000000,3.000000,11.000000,6.666666,12.000000,5.571429,3.000000,17.000000,11.000000,1.000000,...,3.000000,1.000000,0.000000,-1.000000,0.757949,0.916667,0.859582,0.916667,1.000000,-1.000000
max,173.000000,19.000000,68.000000,29.500000,63.000000,105.000000,19.000000,94.000000,62.000000,58.000000,...,24.000000,31.000000,30.000000,29.000000,0.869701,1.000000,0.962479,1.000000,1.000000,1.000000


In [31]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15367 entries, 0 to 15366
Data columns (total 80 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Querylength                      15367 non-null  int64  
 1   domain_token_count               15367 non-null  int64  
 2   path_token_count                 15367 non-null  int64  
 3   avgdomaintokenlen                15367 non-null  float64
 4   longdomaintokenlen               15367 non-null  int64  
 5   avgpathtokenlen                  15096 non-null  float64
 6   tld                              15367 non-null  int64  
 7   charcompvowels                   15367 non-null  int64  
 8   charcompace                      15367 non-null  int64  
 9   ldl_url                          15367 non-null  int64  
 10  ldl_domain                       15367 non-null  int64  
 11  ldl_path                         15367 non-null  int64  
 12  ldl_filename      

In [32]:
df['URL_Type_obf_Type'].value_counts()

URL_Type_obf_Type
benign      7781
phishing    7586
Name: count, dtype: int64

Comparacion de valores nulos

In [33]:
is_null = df.isna().any()
is_null[is_null]

avgpathtokenlen             True
NumberRate_DirectoryName    True
NumberRate_FileName         True
NumberRate_Extension        True
NumberRate_AfterPath        True
Entropy_DirectoryName       True
Entropy_Filename            True
Entropy_Extension           True
Entropy_Afterpath           True
dtype: bool

Comprobacion de valores infinitos

In [34]:
is_inf = df.isin([np.inf, -np.inf]).any()
is_inf[is_inf]

argPathRatio    True
dtype: bool

Division del conjunto de datos

In [35]:
train_set, val_set, test_set = train_val_test_split(df)

In [36]:
X_train, y_train = remove_labels(train_set, 'URL_Type_obf_Type')
X_val, y_val = remove_labels(val_set, 'URL_Type_obf_Type')
X_test, y_test = remove_labels(test_set, 'URL_Type_obf_Type')

In [37]:
X_train = X_train.drop('argPathRatio', axis=1)
X_val = X_val.drop('argPathRatio', axis=1)
X_test = X_test.drop('argPathRatio', axis=1)

In [39]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

In [40]:
#rellenamos valores nulos
X_train_prep = imputer.fit_transform(X_train)
X_val_prep = imputer.fit_transform(X_val)
X_test_prep = imputer.fit_transform(X_test)

In [41]:
#transformamos el resultado a dataframe de pandas
X_train_prep = pd.DataFrame(X_train_prep, columns=X_train.columns, index=y_train.index)
X_val_prep = pd.DataFrame(X_val_prep, columns=X_val.columns, index=y_val.index)
X_test_prep = pd.DataFrame(X_test_prep, columns=X_test.columns, index=y_test.index)

In [42]:
X_train_prep.head(10)

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_Directoryname,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath
2134,0.0,2.0,6.0,2.000000,2.0,8.666667,2.0,17.0,10.0,0.0,...,2.0,0.0,0.0,-1.0,0.681183,0.827729,0.702637,0.849605,0.000000,-1.000000
9178,0.0,4.0,18.0,3.250000,5.0,1.000000,4.0,18.0,13.0,2.0,...,12.0,3.0,0.0,4.0,0.695232,0.820160,0.682849,0.875578,0.000000,0.778747
13622,0.0,3.0,3.0,6.666666,14.0,4.000000,3.0,1.0,1.0,1.0,...,1.0,0.0,0.0,-1.0,0.836006,0.869991,0.879588,1.000000,0.000000,-1.000000
15182,0.0,3.0,5.0,3.333333,4.0,3.000000,3.0,5.0,2.0,0.0,...,2.0,1.0,0.0,-1.0,0.731804,0.796490,0.796658,1.000000,1.000000,-1.000000
8013,74.0,2.0,13.0,9.500000,17.0,7.875000,2.0,21.0,29.0,26.0,...,4.0,5.0,4.0,3.0,0.653371,0.820569,0.758055,0.714969,0.712215,0.708031
12408,0.0,3.0,4.0,8.333333,19.0,3.750000,3.0,5.0,1.0,0.0,...,2.0,0.0,0.0,-1.0,0.726479,0.789538,0.800705,1.000000,0.000000,-1.000000
509,20.0,2.0,13.0,4.500000,6.0,3.000000,2.0,24.0,17.0,0.0,...,1.0,14.0,13.0,12.0,0.678515,0.796658,0.871049,0.695112,0.701662,0.698106
10714,0.0,3.0,8.0,6.666666,14.0,4.250000,3.0,11.0,5.0,0.0,...,4.0,0.0,0.0,-1.0,0.745348,0.869991,0.788921,1.000000,0.000000,-1.000000
3986,0.0,2.0,6.0,6.500000,10.0,4.500000,2.0,7.0,7.0,0.0,...,2.0,0.0,0.0,-1.0,0.760843,0.798231,0.822491,0.796670,0.000000,-1.000000
748,0.0,2.0,8.0,4.000000,5.0,5.750000,2.0,14.0,14.0,1.0,...,2.0,0.0,0.0,-1.0,0.709062,0.929897,0.884735,0.674994,0.000000,-1.000000


In [43]:
#comprobamos si hay valores nulos en el conjunto de datos de entrenamiento
is_null = X_train_prep.isna().any()
is_null[is_null]

Series([], dtype: bool)

Random Forests

In [44]:
from sklearn.ensemble import RandomForestClassifier
clf_rnd = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=1)
clf_rnd.fit(X_train_prep, y_train)

RandomForestClassifier(n_estimators=50, n_jobs=1, random_state=42)

In [45]:
#predecimos con le conjunto de datos de verificacion
y_pred = clf_rnd.predict(X_val_prep)

In [46]:
print('F1 score: ', f1_score(y_pred, y_val, average='weighted'))

F1 score:  0.9814512561669243


Importancia de las caracteristicas

In [47]:
clf_rnd.feature_importances_

array([2.32462451e-03, 4.10883620e-02, 1.55615909e-02, 7.99712479e-03,
       1.65990915e-02, 9.86295461e-03, 7.65844270e-02, 1.07038920e-02,
       4.10909064e-03, 1.71730040e-02, 1.31761281e-04, 1.18464909e-02,
       1.17182622e-03, 2.74032236e-03, 6.25150118e-03, 2.19849025e-05,
       3.33271980e-03, 5.98320411e-04, 1.55665617e-03, 2.92946733e-02,
       6.72526376e-02, 2.72524872e-02, 3.16335950e-02, 3.57637635e-03,
       4.99154337e-03, 2.42803032e-03, 3.75055480e-02, 7.73454936e-03,
       2.33298294e-02, 1.02543054e-01, 5.54922207e-02, 0.00000000e+00,
       0.00000000e+00, 4.17776815e-02, 0.00000000e+00, 2.02031295e-02,
       1.23596524e-03, 8.18909100e-03, 7.63794842e-04, 2.69532057e-03,
       2.86146474e-03, 5.32465161e-03, 1.24179242e-03, 4.43449217e-03,
       3.09250094e-02, 1.61720220e-02, 6.45644241e-03, 1.28759361e-02,
       1.00550211e-03, 3.51679560e-02, 4.54157685e-03, 4.74117594e-03,
       6.42814877e-03, 1.81472771e-03, 2.45274853e-04, 1.06004044e-03,
      

In [48]:
#podemos extraer las caracteristicas que son mas importantes para la clasificacion
feature_importances = {name: score for name, score in zip(list(df), clf_rnd.feature_importances_)}

In [50]:
feature_importances_sorted = pd.Series(feature_importances).sort_values(ascending=False)
feature_importances_sorted.head(20)

domainUrlRatio             0.102543
tld                        0.076584
domainlength               0.067253
SymbolCount_URL            0.064566
pathDomainRatio            0.055492
isPortEighty               0.041778
domain_token_count         0.041088
pathurlRatio               0.037506
Query_LetterCount          0.035168
subDirLen                  0.031634
URL_Letter_Count           0.030925
urlLen                     0.029295
pathLength                 0.027252
argDomanRatio              0.023330
ISIpAddressInDomainName    0.020203
ldl_url                    0.017173
delimeter_Domain           0.016791
longdomaintokenlen         0.016599
host_letter_count          0.016172
path_token_count           0.015562
dtype: float64

Reduccion del numero de caracteristicas

In [51]:
#extraemos las 20 caracteristicas con mas relevancia para el algoritmo
columns = list(feature_importances_sorted.head(20).index)

In [52]:
columns

['domainUrlRatio',
 'tld',
 'domainlength',
 'SymbolCount_URL',
 'pathDomainRatio',
 'isPortEighty',
 'domain_token_count',
 'pathurlRatio',
 'Query_LetterCount',
 'subDirLen',
 'URL_Letter_Count',
 'urlLen',
 'pathLength',
 'argDomanRatio',
 'ISIpAddressInDomainName',
 'ldl_url',
 'delimeter_Domain',
 'longdomaintokenlen',
 'host_letter_count',
 'path_token_count']

In [53]:
X_train_reduced = X_train[columns].copy()
X_val_reduced = X_val[columns].copy()

In [55]:
X_train_reduced.head(15)

,domainUrlRatio,tld,domainlength,SymbolCount_URL,pathDomainRatio,isPortEighty,domain_token_count,pathurlRatio,Query_LetterCount,subDirLen,URL_Letter_Count,urlLen,pathLength,argDomanRatio,ISIpAddressInDomainName,ldl_url,delimeter_Domain,longdomaintokenlen,host_letter_count,path_token_count
2134,0.072464,2,5,5,11.400000,-1,2,0.826087,-1,57,60,69,57,0.400000,-1,0,0,2,4,6
9178,0.166667,4,16,20,4.562500,-1,4,0.760417,0,73,69,96,73,4.250000,-1,2,0,5,13,18
13622,0.511628,3,22,5,0.636364,-1,3,0.325581,-1,14,30,43,14,0.090909,-1,1,0,14,20,3
15182,0.315789,3,12,7,1.583333,-1,3,0.500000,-1,19,29,38,19,0.166667,-1,0,0,4,10,5
8013,0.107527,2,20,12,7.950000,-1,2,0.854839,31,159,107,186,159,4.400000,-1,26,0,17,19,13
12408,0.509434,3,27,6,0.703704,-1,3,0.358491,-1,19,44,53,19,0.074074,-1,0,0,19,25,4
509,0.088496,2,10,18,9.600000,-1,2,0.849557,14,96,82,113,96,8.000000,-1,0,0,6,9,13
10714,0.314286,3,22,8,1.863636,-1,3,0.585714,-1,41,54,70,41,0.090909,-1,0,0,14,20,8
3986,0.264151,2,14,5,2.285714,-1,2,0.603774,-1,32,40,53,32,0.142857,-1,0,0,10,13,6
748,0.126761,2,9,5,6.000000,-1,2,0.760563,-1,54,54,71,54,0.222222,-1,1,0,5,8,8


In [56]:
clf_rnd = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=1)
clf_rnd.fit(X_train_reduced, y_train)

RandomForestClassifier(n_estimators=50, n_jobs=1, random_state=42)

In [57]:
y_pred = clf_rnd.predict(X_val_reduced)

In [58]:
print('F1 score: ', f1_score(y_pred, y_val, average='weighted'))

F1 score:  0.964865700544521
